### Como manipular XML com XPath

In [5]:
from gensim.models import KeyedVectors
import unicodedata

import gzip
import xml.etree.ElementTree as ET

with gzip.open('pordesc2018-small.xml.gz') as pordesc2018:
    tree = ET.parse(pordesc2018)
    

### Monta Dicionario

In [6]:
from unicodedata import normalize
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

In [7]:
wordModel = KeyedVectors.load_word2vec_format('health_w2v_unigram_50.bin', binary=True)

dictMesh = {}
for d in tree.findall("./DescriptorRecord"):
    terms = []
    
    for aql in d.findall('.AllowableQualifiersList/AllowableQualifier/QualifierReferredTo/QualifierName'):
        teste_qualifier = aql.find('./String').text
        if(teste_qualifier == 'anatomy & histology' or teste_qualifier == 'pharmacology' or teste_qualifier == 'methods' or teste_qualifier == 'diagnosis'):
            qualifier = teste_qualifier
            break
        else:
            qualifier = "#"
    
    for c in d.findall('.ConceptList/'):
        if c.find('./ScopeNote') != None:
            scope = c.find('./ScopeNote').text.replace('\n','').strip()
        for t in c.findall('./TermList/'):
            terms.append(t.find('./String').text)
            
            palavra_similar = []
            
            if t.find('./String').text.lower() in wordModel.vocab:
                sem_assento = remover_acentos(t.find('./String').text)
                palavra_similar = wordModel.most_similar_cosmul(sem_assento.lower(),topn=10)
                for similar, porcentagem in palavra_similar:
                    if(porcentagem > 0.9):
                        terms.append("<i>"+similar+"</i>")

            
    dictMesh[d.find('.DescriptorUI').text] = {
        'ID': d.find('.DescriptorUI').text,
        'name': d.find('.DescriptorName/String').text,
        'scope': scope,
        'terms': set(terms),
        'qualifier': qualifier
    }
                
    #break
len(dictMesh)

1859

### Imprime Exemplo

In [8]:
#for i in dictMesh:
#    print (i)

### Salva Dicionario

In [9]:
import gzip, pickle

with gzip.open('dictMesh.dict.gz','wb') as fp:
    pickle.dump(dictMesh,fp)
    fp.close()

### Carrega Dicionario

In [10]:
import gzip, pickle

with gzip.open('dictMesh.dict.gz','rb') as fp:
    dictMesh = pickle.load(fp)
    fp.close()
    
dictMesh['D000069280']

{'ID': 'D000069280',
 'name': 'Eletrocorticografia[Electrocorticography]',
 'scope': 'Recording of brain electrical activities in which the electrodes are placed directly on the CEREBRAL CORTEX.',
 'terms': {'ECoG, Extraoperative',
  'ECoG, Intraoperative',
  'ECoGs, Extraoperative',
  'ECoGs, Intraoperative',
  'EEG, Intracranial',
  'EEGs, Intracranial',
  'Electrocorticographies',
  'Electrocorticographies (EcoG)',
  'Electrocorticographies, Extraoperative',
  'Electrocorticographies, Intraoperative',
  'Electrocorticography',
  'Electrocorticography (EcoG)',
  'Electrocorticography, Extraoperative',
  'Electrocorticography, Intraoperative',
  'Electroencephalographies, Intracranial',
  'Electroencephalography, Intracranial',
  'Eletrocorticografia',
  'Extraoperative ECoG',
  'Extraoperative ECoGs',
  'Extraoperative Electrocorticographies',
  'Extraoperative Electrocorticography',
  'Intracranial EEG',
  'Intracranial EEGs',
  'Intracranial Electroencephalographies',
  'Intracrani

In [ ]:
palavra = 'Red meat'
for dui in dictMesh:
    d = dictMesh[dui]
    for t in d['terms']:
        if t.lower() == palavra.lower():
            print(dui, d['qualifier'])
            break

In [ ]:
termos=" "
confirma = False
evolucao = "abd"
strr = ""

#Verifica a lista para ver se a palavra esta no dicionario
cont = 0
for palavra in evolucao:
    for dui in dictMesh:
        d = dictMesh[dui]
        for t in d['terms']:
            if t.lower() == palavra.lower():
                evolucao[cont] = '<a href="#" data-ui="das" data-term="" data-scope="'+d['scope']+'">'+palavra+'</a>'
                cont +=1

    teste = dictMesh[dui]['terms']
    termos = ' '.join(teste)
evolucao = evolucao.replace('data-terms=""', 'data-terms="'+termos+'"')#pegar os termos da lista e por aqui
strr += ' '+evolucao

#print(termos)
termos

In [ ]:
d['terms']

In [ ]:
strr = "meu[nome]"
strr = strr.replace('[', ' [')
print(strr)
